<a href="https://colab.research.google.com/github/sileod/tasknet/blob/main/Tasknet_multitask_transformer_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.system('pip install tasknet')
import tasknet as tn
from datasets import load_dataset

In [ ]:
import tasknet as tn


class args:
    mtl_architecture = "cls_task_embedding"
    model_name = "roberta-base"
    
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = 3e-5
    max_steps = 200


rte = tn.Classification(
    dataset=load_dataset("glue", "rte"), s1="sentence1", s2="sentence2", y="label"
)


conll_pos = tn.TokenClassification(
    dataset="conll2003", # A dataset string/tuple can also be passed and loaded internally with datasets.load_dataset
    name="conll_pos", tokens="tokens", y="pos_tags"
)

reco = tn.MultipleChoice(
    tn.utils.train_validation_test_split(
        tn.load_dataset("sileod/movie_recommendation")["test"]
    ),
    s1="question",
    y="label",
    choices=["answer_0", "answer_1", "answer_2", "answer_3"],
)

stsb = tn.Classification(
    dataset=("glue", "stsb"), s1="sentence1", s2="sentence2", y="label"
)



# Multi-task learning

In [ ]:
tasks = [rte, conll_pos, reco, stsb]
models = tn.Model(tasks, args) # list of models; by default, shared encoder, task-specific CLS token task-specific head 
trainer = tn.Trainer(models, tasks, args) # tasks are uniformly sampled
trainer.train()